# How to visualise data with xcube Viewer in your workspace
**A DeepESDL example notebook**

This notebook demonstrates how to use the features of the xcube JupyterLab integration.
The notebook demonstrates three scenarios how xcube Viewer is utilized in JupyterLab.
In particular, we open xcube Viewer for any `xarray.Dataset` instances 

1. persisted in deepesdl public s3 storage (saved datasets)
2. opened or otherwise created in this Notebook (in-memory datasets)
3. using a configuration file for customising styles. 

To explore the xcube viewer functionalities, please checkout the documentation: https://xcube.readthedocs.io/en/latest/viewer.html#functionality

Please, also refer to the [DeepESDL documentation](https://earthsystemdatalab.net/guide/jupyterlab) and visit the platform's [website](https://www.earthsystemdatalab.net/) for further information!

Brockmann Consult, 2025

-----------------

**This notebook runs with the python environment `users-deepesdl-xcube-1.11.0`, please checkout the documentation for [help on changing the environment](https://earthsystemdatalab.net/guide/jupyterlab/#python-environment-selection-of-the-jupyter-kernel).**

For example reasons we will access a deepesdl public data cube and visualize it in the jupyter lab using xcube viewer

In [1]:
from xcube.core.store import new_data_store
store = new_data_store("s3", root="deep-esdl-public")
store.list_data_ids()

['LC-1x2025x2025-2.0.0.levels',
 'LC-1x2160x2160-1.0.0.levels',
 'SMOS-L2C-OS-20230101-20231231-1W-res0-1x1000x1000.levels',
 'SMOS-L2C-OS-20230101-20231231-1W-res0-53x120x120.zarr',
 'SMOS-L2C-OS-20230101-20231231-1W-res0.zarr',
 'SMOS-L2C-SM-20230101-20231231-1W-res0-1x1000x1000.levels',
 'SMOS-L2C-SM-20230101-20231231-1W-res0-53x120x120.zarr',
 'SMOS-L2C-SM-20230101-20231231-1W-res0.zarr',
 'SMOS-freezethaw-1x720x720-1.0.1.zarr',
 'SMOS-freezethaw-4267x10x10-1.0.1.zarr',
 'SeasFireCube-8D-0.25deg-1x720x1440-3.0.0.zarr',
 'SeasFireCube-8D-0.25deg-966x180x360-3.0.0.zarr',
 'SeasFireCube_v3.zarr',
 'black-sea-1x1024x1024.levels',
 'black-sea-256x128x128.zarr',
 'cmems_sst_v2.zarr',
 'esa-cci-permafrost-1x1151x1641-0.0.2.levels',
 'esa-cci-permafrost-1x1151x1641-1.0.0.zarr',
 'esdc-8d-0.083deg-184x270x270-2.1.1.zarr',
 'esdc-8d-0.25deg-1x720x1440-3.0.1.zarr',
 'esdc-8d-0.25deg-256x128x128-3.0.1.zarr',
 'extrAIM-merged-cube-1x86x179.zarr',
 'hydrology-1D-0.009deg-100x60x60-3.0.2.zarr',
 

We use the xcube datastore framework here to open the dataset, but it could also be opened by other means, e.g., `xr.open_dataset()`, provided it has variables with dimensions ["time", "y", "x"] or ["y", "x"]. 

Let's visualize the Earth System Data Cube:

In [2]:
dataset = store.open_data("esdc-8d-0.25deg-1x720x1440-3.0.1.zarr")

In [3]:
dataset

<xarray.Dataset> Size: 353GB
Dimensions:                            (time: 1978, lat: 720, lon: 1440)
Coordinates:
  * lat                                (lat) float64 6kB -89.88 -89.62 ... 89.88
  * lon                                (lon) float64 12kB -179.9 ... 179.9
  * time                               (time) datetime64[ns] 16kB 1979-01-05 ...
Data variables: (12/42)
    aerosol_optical_thickness_550      (time, lat, lon) float32 8GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    air_temperature_2m                 (time, lat, lon) float32 8GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    bare_soil_evaporation              (time, lat, lon) float32 8GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    burnt_area                         (time, lat, lon) float64 16GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    cot                                (time, lat, lon) float32 8GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    cth                                (time, lat, lon) float32 8GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    ...                                 ...
    sif_rtsif                          (time, lat, lon) float32 8GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    sm                                 (time, lat, lon) float32 8GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    snow_sublimation                   (time, lat, lon) float32 8GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    surface_moisture                   (time, lat, lon) float32 8GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    terrestrial_ecosystem_respiration  (time, lat, lon) float32 8GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    transpiration                      (time, lat, lon) float32 8GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
Attributes: (12/23)
    Conventions:                CF-1.9
    acknowledgment:             All ESDC data providers are acknowledged insi...
    contributor_name:           ['University of Leipzig', 'Max Planck Institu...
    contributor_url:            ['https://www.uni-leipzig.de/', 'https://www....
    creator_name:               ['University of Leipzig', 'Brockmann Consult ...
    creator_url:                ['https://www.uni-leipzig.de/', 'https://www....
    ...                         ...
    publisher_url:              https://www.earthsystemdatalab.net/
    time_coverage_end:          2021-12-31T00:00:00.000000000
    time_coverage_start:        1979-01-05T00:00:00.000000000
    time_period:                8D
    time_period_reported_day:   5.0
    title:                      Earth System Data Cube (ESDC) v3.0.1

In [4]:
from xcube.webapi.viewer import Viewer

---
## Scenario 1: Open xcube Viewer for a persisted datasets

Open xcube Viewer for a dataset instances persisted from a certain source (saved datasets).

**For the functionalities of xcube viewer please head over to the documentation:** https://xcube.readthedocs.io/en/latest/viewer.html#functionality

In [5]:
viewer = Viewer()

404 GET /viewer/config/config.json (127.0.0.1): xcube viewer has not been been configured
404 GET /viewer/config/config.json (127.0.0.1) 3.24ms
501 GET /viewer/state?key=sentinel (127.0.0.1) 0.42ms
404 GET /viewer/ext/contributions (127.0.0.1) 68.60ms


In [6]:
viewer.add_dataset(dataset)

'4ae13077-8b73-4765-83e4-93adf6dc8cf7'

You can click on the viewer link to open xcube Viewer in a new browser tab:

In [7]:
viewer.info()

Server: https://deep.earthsystemdatalab.net/user/clarasb/proxy/8000
Viewer: https://deep.earthsystemdatalab.net/user/clarasb/proxy/8000/viewer/?serverUrl=https://deep.earthsystemdatalab.net/user/clarasb/proxy/8000


You can also open xcube Viewer inlined here:

In [8]:
viewer.show()

To stop the server and viewer:

In [9]:
viewer.stop_server()

---
## **Scenario 2**: Open xcube Viewer for in-memory datasets

Open xcube Viewer for a dataset instances opened or otherwise created in this Notebook (in-memory datasets).

Below, let's fetch a dataset from the [Open Data Portal](https://climate.esa.int/en/odp/#/dashboard) of the [ESA Climate Change Initiative](https://climate.esa.int/en/esa-climate/esa-cci/) (CCI) on the fly, without persisting it - for more details about the xcube CCI datastore please checkout the example notebook in xcube-datastores *GENERATE CCI CUBES*. Be aware of performance loss, so in case you plan to use a dataset a lot for analysis or visualisation, please persist it into the team s3 storage. The public cubes provided within DeepESDL are already persisted in S3, so you should not duplicate them in your team storage. 

In [14]:
cci_store = new_data_store('cciodp')

In [11]:
bbox=[-5, 45, 35, 65]

In [12]:
cci_dataset = cci_store.open_data(
    "esacci.SST.day.L4.SSTdepth.multi-sensor.multi-platform.OSTIA.2-1.sst",
    bbox=bbox,
    variable_names=["analysed_sst"],
    time_range=["2015-01-30", "2016-12-31"],
)

cci_dataset

<xarray.Dataset> Size: 2GB
Dimensions:       (time: 702, lat: 400, lon: 800, bnds: 2)
Coordinates:
  * lat           (lat) float32 2kB 45.03 45.08 45.12 ... 64.88 64.93 64.97
    lat_bnds      (lat, bnds) float32 3kB dask.array<chunksize=(400, 2), meta=np.ndarray>
  * lon           (lon) float32 3kB -4.975 -4.925 -4.875 ... 34.88 34.92 34.97
    lon_bnds      (lon, bnds) float32 6kB dask.array<chunksize=(800, 2), meta=np.ndarray>
  * time          (time) datetime64[ns] 6kB 2015-01-30T12:00:00 ... 2016-12-3...
    time_bnds     (time, bnds) datetime64[ns] 11kB dask.array<chunksize=(702, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    analysed_sst  (time, lat, lon) float64 2GB dask.array<chunksize=(1, 400, 800), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7
    title:                   esacci.SST.day.L4.SSTdepth.multi-sensor.multi-pl...
    date_created:            2025-09-23T10:13:43.985305
    processing_level:        L4
    time_coverage_start:     2015-01-30T00:00:00
    time_coverage_end:       2017-01-01T00:00:00
    time_coverage_duration:  P702DT0H0M0S
    history:                 [{'program': 'xcube_cci.chunkstore.CciChunkStore...

In [15]:
cci_dataset.attrs["title"] = "CCI SST"


Let's set some attributes of the variable, so the colormapping does not fall back on the default

In [16]:
cci_dataset.analysed_sst.attrs["color_value_min"] = 270
cci_dataset.analysed_sst.attrs["color_value_max"] = 310
cci_dataset.analysed_sst.attrs["color_bar_name"] = "plasma"

In [17]:
viewer = Viewer()

404 GET /viewer/config/config.json (127.0.0.1): xcube viewer has not been been configured
404 GET /viewer/config/config.json (127.0.0.1) 2.94ms
501 GET /viewer/state?key=sentinel (127.0.0.1) 0.27ms
404 GET /viewer/ext/contributions (127.0.0.1) 0.47ms


In [18]:
viewer.add_dataset(dataset)
viewer.add_dataset(cci_dataset)

'03305023-bae1-4644-a5e6-d308248d4c72'

You can click on the viewer link to open xcube Viewer in a new browser tab:

In [19]:
viewer.info()

Server: https://deep.earthsystemdatalab.net/user/clarasb/proxy/8001
Viewer: https://deep.earthsystemdatalab.net/user/clarasb/proxy/8001/viewer/?serverUrl=https://deep.earthsystemdatalab.net/user/clarasb/proxy/8001


You can also open xcube Viewer inlined here:

In [20]:
viewer.show()

To stop the server and viewer:

In [21]:
viewer.stop_server()

---

## **Scenario 3**: Use custom a server configuration 

Use custom server configuration to start server and pass it to the viewer constructor. In this example, we have create a local file with the configuration and load it as a dictionary and pass it to the viewer. 

The configuration file is pointing to your team storage as input source. If you don't have anything stored in your team S3 storage, the viewer will not be able to display any data. Please check other example notebooks and create some sample datasets in your team storage to visualize the content. 

The custom configuration allows you to predefined your value ranges, the colormaps that should be used as well as which bands should be used to create an RGB image, then the RGB switch in the viewer will display the RGB image. 

If you do not have a server-config.yaml file in your directory, please create one with the following content: 

```yaml
DataStores:
  - Identifier: deep-esdl-team-storage
    StoreId: s3
    StoreParams:
      root: $S3_USER_STORAGE_BUCKET
      storage_options:
          anon: false
          key: $S3_USER_STORAGE_KEY
          secret: $S3_USER_STORAGE_SECRET
    Datasets:
      - Path: "*.zarr"
        Style: default

        # ChunkCacheSize: 1G


Styles:
  - Identifier: default
    ColorMappings:
      analysed_sst:
        ColorBar: plasma
        ValueRange: [270, 310]
## if you have bands that can create an RGB image, you can specify them as below. 
#      rgb:
#        Red:
#          Variable: B04
#          ValueRange: [0., 0.25]
#        Green:
#          Variable: B03
#          ValueRange: [0., 0.25]
#        Blue:
#          Variable: B02
#          ValueRange: [0., 0.25]

```

**For all possible settings within the server configuration file, please checkout the documentation:** https://xcube.readthedocs.io/en/latest/cli/xcube_serve.html#configuration-file

In [32]:
from xcube.util.config import load_configs

In [ ]:
viewer = Viewer(server_config=load_configs("server-config.yaml"))

In [36]:
viewer.show()

To stop the server and viewer:

In [37]:
viewer.stop_server()